In [5]:
# Object manipulation
import statistics
import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report,mean_squared_error
from sklearn.tree import plot_tree
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
rs = 24
from pandas.tseries.holiday import * 

print("✅ Libaries import successfully")


✅ Libaries import successfully


In [6]:
df = pd.read_csv(r"C:\Users\Akshat Sharma\OneDrive\Desktop\Data Science\Machine learning\Decision Tree\Imarticus Dataset\Decision Tree Project\Dataset 2\Random Forest Project\Dataset\TaxiFare.csv",parse_dates= ["date_time_of_pickup"] )

In [7]:
df.head()

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
0,26:21.0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [8]:
df.shape

(50000, 8)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             50000 non-null  object             
 1   amount                50000 non-null  float64            
 2   date_time_of_pickup   50000 non-null  datetime64[ns, UTC]
 3   longitude_of_pickup   50000 non-null  float64            
 4   latitude_of_pickup    50000 non-null  float64            
 5   longitude_of_dropoff  50000 non-null  float64            
 6   latitude_of_dropoff   50000 non-null  float64            
 7   no_of_passenger       50000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 3.1+ MB


# Converting UTC time to Eastern Standard Time (US)

### Module - import pytz

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             50000 non-null  object             
 1   amount                50000 non-null  float64            
 2   date_time_of_pickup   50000 non-null  datetime64[ns, UTC]
 3   longitude_of_pickup   50000 non-null  float64            
 4   latitude_of_pickup    50000 non-null  float64            
 5   longitude_of_dropoff  50000 non-null  float64            
 6   latitude_of_dropoff   50000 non-null  float64            
 7   no_of_passenger       50000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 3.1+ MB


In [11]:
import pytz

# Define the target timezone
est_zone = pytz.timezone('US/Eastern')

# Convert timestamps to EST
df['date_time_of_pickup'] = df['date_time_of_pickup'].dt.tz_convert(est_zone)

In [12]:
df.head()

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
0,26:21.0,4.5,2009-06-15 13:26:21-04:00,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 11:52:16-05:00,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-17 20:35:00-04:00,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 00:30:42-04:00,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 02:51:00-05:00,-73.968095,40.768008,-73.956655,40.783762,1


## Extracting use full thinks from date_time_of_pickup column

In [13]:
df["year"] = df["date_time_of_pickup"].dt.year
df["month"] = df["date_time_of_pickup"].dt.month
df["day_of_week"] = df["date_time_of_pickup"].dt.dayofweek

In [14]:
import holidays
ny_holiday = holidays.US()

In [15]:
def holiday(x):
    
    if (x in ny_holiday) | (x.dayofweek > 4):
        return 1
    else:
        return 0

In [16]:
df["is_holiday"] = df["date_time_of_pickup"].apply(lambda x: holiday(x))

In [17]:
df["is_holiday"].value_counts()

is_holiday
0    35770
1    14230
Name: count, dtype: int64

In [18]:
df.head()

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger,year,month,day_of_week,is_holiday
0,26:21.0,4.5,2009-06-15 13:26:21-04:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,0
1,52:16.0,16.9,2010-01-05 11:52:16-05:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,0
2,35:00.0,5.7,2011-08-17 20:35:00-04:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,2,0
3,30:42.0,7.7,2012-04-21 00:30:42-04:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,1
4,51:00.0,5.3,2010-03-09 02:51:00-05:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,0


In [19]:
df["hour"] = df["date_time_of_pickup"].dt.hour
df["minute"] = df["date_time_of_pickup"].dt.minute

In [20]:
df

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger,year,month,day_of_week,is_holiday,hour,minute
0,26:21.0,4.5,2009-06-15 13:26:21-04:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,0,13,26
1,52:16.0,16.9,2010-01-05 11:52:16-05:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,0,11,52
2,35:00.0,5.7,2011-08-17 20:35:00-04:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,2,0,20,35
3,30:42.0,7.7,2012-04-21 00:30:42-04:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,1,0,30
4,51:00.0,5.3,2010-03-09 02:51:00-05:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,0,2,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,25:15.0,15.0,2013-06-12 19:25:15-04:00,-73.999973,40.748531,-74.016899,40.705993,1,2013,6,2,0,19,25
49996,19:18.0,7.5,2015-06-22 13:19:18-04:00,-73.984756,40.768211,-73.987366,40.760597,1,2015,6,0,0,13,19
49997,53:00.0,6.9,2011-01-29 23:53:00-05:00,-74.002698,40.739428,-73.998108,40.759483,1,2011,1,5,1,23,53
49998,09:00.0,4.5,2012-11-06 02:09:00-05:00,-73.946062,40.777567,-73.953450,40.779687,2,2012,11,1,0,2,9


In [21]:
df.groupby("hour")["amount"].mean()

hour
0     14.302599
1     13.748498
2     11.330138
3     11.088714
4     10.487339
5     10.567240
6     11.202583
7     11.038858
8     11.404517
9     11.920202
10    11.799757
11    12.152969
12    12.206699
13    11.116186
14    11.053722
15    10.879833
16    10.667583
17    11.014533
18    11.341762
19    11.319136
20    11.961027
21    11.698287
22    11.679353
23    12.152016
Name: amount, dtype: float64

In [22]:
df.loc[5686, "latitude_of_pickup"] = 40.1083332

In [23]:
df.loc[5686]

unique_id                                 15:00.0
amount                                        3.3
date_time_of_pickup     2011-07-30 07:15:00-04:00
longitude_of_pickup                    -73.947235
latitude_of_pickup                      40.108333
longitude_of_dropoff                   -73.951392
latitude_of_dropoff                     40.778927
no_of_passenger                                 1
year                                         2011
month                                           7
day_of_week                                     5
is_holiday                                      1
hour                                            7
minute                                         15
Name: 5686, dtype: object

## Calculating Distance 

In [24]:
import pandas as pd
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

In [25]:
# Calculate distances using the haversine formula
df['distance_km'] = haversine_distance(df['latitude_of_pickup'], df['longitude_of_pickup'], 
                                        df['latitude_of_dropoff'], df['longitude_of_dropoff'])

In [26]:
df

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger,year,month,day_of_week,is_holiday,hour,minute,distance_km
0,26:21.0,4.5,2009-06-15 13:26:21-04:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,0,13,26,1.030764
1,52:16.0,16.9,2010-01-05 11:52:16-05:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,0,11,52,8.450134
2,35:00.0,5.7,2011-08-17 20:35:00-04:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,2,0,20,35,1.389525
3,30:42.0,7.7,2012-04-21 00:30:42-04:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,1,0,30,2.799270
4,51:00.0,5.3,2010-03-09 02:51:00-05:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,0,2,51,1.999157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,25:15.0,15.0,2013-06-12 19:25:15-04:00,-73.999973,40.748531,-74.016899,40.705993,1,2013,6,2,0,19,25,4.940374
49996,19:18.0,7.5,2015-06-22 13:19:18-04:00,-73.984756,40.768211,-73.987366,40.760597,1,2015,6,0,0,13,19,0.874706
49997,53:00.0,6.9,2011-01-29 23:53:00-05:00,-74.002698,40.739428,-73.998108,40.759483,1,2011,1,5,1,23,53,2.263286
49998,09:00.0,4.5,2012-11-06 02:09:00-05:00,-73.946062,40.777567,-73.953450,40.779687,2,2012,11,1,0,2,9,0.665245


In [27]:
rev_ind = df[df["distance_km"] == 0].index

In [28]:
df.drop(index = rev_ind, inplace = True)

In [29]:
df.shape

(48551, 15)

In [30]:
df.columns

Index(['unique_id', 'amount', 'date_time_of_pickup', 'longitude_of_pickup',
       'latitude_of_pickup', 'longitude_of_dropoff', 'latitude_of_dropoff',
       'no_of_passenger', 'year', 'month', 'day_of_week', 'is_holiday', 'hour',
       'minute', 'distance_km'],
      dtype='object')

In [35]:
def get_hour(x):
    hour = x["hour"]
    minute = x["minute"]
    if minute <= 15:
        return hour
    elif (minute > 15) & (minute <= 45):
        return hour + 0.5
    else:
        return hour + 1

In [37]:
df["approx_time"] = df.apply(lambda x: get_hour(x), axis = 1)

In [38]:
df.columns

Index(['unique_id', 'amount', 'date_time_of_pickup', 'longitude_of_pickup',
       'latitude_of_pickup', 'longitude_of_dropoff', 'latitude_of_dropoff',
       'no_of_passenger', 'year', 'month', 'day_of_week', 'is_holiday', 'hour',
       'minute', 'distance_km', 'approx_time'],
      dtype='object')

In [41]:
df.drop(index = df[df["amount"] < 0].index, inplace = True)

In [42]:
df.shape

(48545, 16)

In [45]:
df.drop(index = df[df["amount"] <= 0].index, inplace = True)

In [46]:
df.shape

(48542, 16)

In [49]:
df.columns

Index(['unique_id', 'amount', 'date_time_of_pickup', 'longitude_of_pickup',
       'latitude_of_pickup', 'longitude_of_dropoff', 'latitude_of_dropoff',
       'no_of_passenger', 'year', 'month', 'day_of_week', 'is_holiday', 'hour',
       'minute', 'distance_km', 'approx_time'],
      dtype='object')

# Spliting

In [67]:
X = df.drop(columns = ['unique_id', 'amount', 'date_time_of_pickup'])

y = df["amount"]

In [68]:
X.columns

Index(['longitude_of_pickup', 'latitude_of_pickup', 'longitude_of_dropoff',
       'latitude_of_dropoff', 'no_of_passenger', 'year', 'month',
       'day_of_week', 'is_holiday', 'hour', 'minute', 'distance_km',
       'approx_time'],
      dtype='object')

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=rs)

In [70]:
X_train.shape, y_train.shape

((38833, 13), (38833,))

# Base Model

In [71]:
model = RandomForestRegressor(random_state=rs)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [72]:
model.score(X_test, y_test)

0.8297464846595868

In [ ]:
model.score(X_test, y_test)

0.7835365237758488

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7835365237758488

In [73]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8297464846595868

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.485051324150669

In [59]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.518339718987705

In [ ]:
model.feature_importances_

array([0.0096355 , 0.03133019, 0.02702726, 0.01664311, 0.00373214,
       0.03781827, 0.03430013, 0.8395134 ])

In [60]:
model.feature_importances_

array([0.01099564, 0.03494375, 0.02797008, 0.01948829, 0.00342916,
       0.85738628, 0.0457868 ])

In [ ]:
imp_df = pd.DataFrame({"Name": X_train.columns, "Imp": model.feature_importances_})

imp_df.sort_values(by = ["Imp"],ascending=False, inplace=True)

In [ ]:
imp_df

,Name,Imp
7,distance_km,0.839513
5,hour,0.037818
6,minute,0.034300
1,year,0.031330
2,month,0.027027
3,day_of_week,0.016643
0,no_of_passenger,0.009635
4,is_holiday,0.003732


In [74]:
imp_df = pd.DataFrame({"Name": X_train.columns, "Imp": model.feature_importances_})

imp_df.sort_values(by = ["Imp"],ascending=False, inplace=True)

In [75]:
imp_df

,Name,Imp
11,distance_km,0.796686
2,longitude_of_dropoff,0.043035
0,longitude_of_pickup,0.029953
3,latitude_of_dropoff,0.027504
5,year,0.025818
1,latitude_of_pickup,0.021151
10,minute,0.013420
12,approx_time,0.013138
6,month,0.009980
9,hour,0.007445


In [ ]:
# import requests
# import pandas as pd

# # Replace 'YOUR_API_KEY' with your actual MapQuest API key
# api_key = 'xlCqcT6XbeFMfIhnadur0Z0ZRInxFjqn'

# # Initialize an empty list to store distances
# distances = []

# # Iterate through the DataFrame rows and calculate distances
# for index, row in df.iterrows():
#     pickup = f"{row['latitude_of_pickup']},{row['longitude_of_pickup']}"
#     dropoff = f"{row['latitude_of_dropoff']},{row['longitude_of_dropoff']}"
    
#     url = f"https://www.mapquestapi.com/directions/v2/route?key={api_key}&from={pickup}&to={dropoff}&routeType=fastest"
#     response = requests.get(url)
#     data = response.json()
    
#     if data.get("route"):
#         distance = data["route"]["distance"]
#         distances.append(distance)

# # Add the distances to the DataFrame
# df['driving_distance_km'] = distances

# print(df)


In [ ]:
# import requests
# import pandas as pd

# # Replace 'YOUR_API_KEY' with your actual MapQuest API key
# api_key = 'xlCqcT6XbeFMfIhnadur0Z0ZRInxFjqn'



# # Initialize an empty list to store distances
# distances = []

# # Iterate through the DataFrame rows and calculate distances
# for index, row in df.iterrows():
#     pickup = f"{row['latitude_of_pickup']},{row['longitude_of_pickup']}"
#     dropoff = f"{row['latitude_of_dropoff']},{row['longitude_of_dropoff']}"
    
#     url = f"https://www.mapquestapi.com/directions/v2/route?key={api_key}&from={pickup}&to={dropoff}&routeType=fastest"
#     response = requests.get(url)
#     data = response.json()
    
#     if data.get("route"):
#         distance = data["route"]["distance"]
#         distances.append(distance)

# # Add the distances to the DataFrame
# df['driving_distance_km'] = distances

# print(df)


## Applying Cross Validation

In [63]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [79]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits = 3, shuffle=False)

# recall score of each split
score = cross_val_score(model, X_train, y_train, scoring='r2', cv=kf)
score

array([0.79807298, 0.82155786, 0.79666569])

In [80]:
score.mean()

0.8054321757770341

## Randomized Search CV

In [78]:
from sklearn.model_selection import RandomizedSearchCV

In [82]:


param_dict = {
    "n_estimators": [100, 200, 250, 300, 350, 400, 500, 600],  # You can add more values
    "criterion": ["squared_error", "absolute_error","poisson", "friedman_mse"],
    "max_depth": [x for x in range(15,40,2)],
    # "min_samples_split": [x for x in range(10,25,2)],
    # "min_samples_leaf": [x for x in range(4,14,2)],
    "max_features": [None, "sqrt"]
}

gs = RandomizedSearchCV(estimator=model, param_distributions= param_dict, n_iter=10,
                        scoring="neg_root_mean_squared_error", cv = kf, verbose=3, n_jobs= -1)
gs.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: 